In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv("F:/movie-recommendation-system/content-based-filtering/movie_dataset.csv")
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
# we can see that there are a lot of columns in the dataset.
# but most of them is useless.
# So, we choose keywords, cast, genres and director column to use as our feature set.
features = ['keywords','cast','genres','director']

In [4]:
#Our next task is to create a function for combining the values of these columns into a single string.

def combine_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']

In [5]:
# we need to clean out our new data by filling all NaN by an empty string ""

for feature in features:
    data[feature] = data[feature].fillna('') 
    
data[features].head(10)

,keywords,cast,genres,director
0,culture clash future space war space colony so...,Sam Worthington Zoe Saldana Sigourney Weaver S...,Action Adventure Fantasy Science Fiction,James Cameron
1,ocean drug abuse exotic island east india trad...,Johnny Depp Orlando Bloom Keira Knightley Stel...,Adventure Fantasy Action,Gore Verbinski
2,spy based on novel secret agent sequel mi6,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Action Adventure Crime,Sam Mendes
3,dc comics crime fighter terrorist secret ident...,Christian Bale Michael Caine Gary Oldman Anne ...,Action Crime Drama Thriller,Christopher Nolan
4,based on novel mars medallion space travel pri...,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Action Adventure Science Fiction,Andrew Stanton
5,dual identity amnesia sandstorm love of one's ...,Tobey Maguire Kirsten Dunst James Franco Thoma...,Fantasy Action Adventure,Sam Raimi
6,hostage magic horse fairy tale musical,Zachary Levi Mandy Moore Donna Murphy Ron Perl...,Animation Family,Byron Howard
7,marvel comic sequel superhero based on comic b...,Robert Downey Jr. Chris Hemsworth Mark Ruffalo...,Action Adventure Science Fiction,Joss Whedon
8,witch magic broom school of witchcraft wizardry,Daniel Radcliffe Rupert Grint Emma Watson Tom ...,Adventure Fantasy Family,David Yates
9,dc comics vigilante superhero based on comic b...,Ben Affleck Henry Cavill Gal Gadot Amy Adams J...,Action Adventure Fantasy,Zack Snyder


In [6]:
data["combined_features"] = data.apply(combine_features,axis=1) 
# applying combined_features() method over each rows of dataframe and storing the combined string in "combined_features" column

data["combined_features"].head(10)

0    culture clash future space war space colony so...
1    ocean drug abuse exotic island east india trad...
2    spy based on novel secret agent sequel mi6 Dan...
3    dc comics crime fighter terrorist secret ident...
4    based on novel mars medallion space travel pri...
5    dual identity amnesia sandstorm love of one's ...
6    hostage magic horse fairy tale musical Zachary...
7    marvel comic sequel superhero based on comic b...
8    witch magic broom school of witchcraft wizardr...
9    dc comics vigilante superhero based on comic b...
Name: combined_features, dtype: object

In [7]:
cv = CountVectorizer() 
#creating new CountVectorizer() object

count_matrix = cv.fit_transform(data["combined_features"]) 
#feeding combined strings(movie contents) to CountVectorizer() object

In [8]:
cosine_sim = cosine_similarity(count_matrix)

In [9]:
# to get title of a movie from a given index
def get_title_from_index(index):
    return data[data.index == index]["title"].values[0]

# to get index of a movie from a given title
def get_index_from_title(title):
    return data[data.title == title]["index"].values[0]

In [10]:
# Our next step is to get the title of the movie that the user currently likes. 
# Then we will find the index of that movie. 
# After that, we will access the row corresponding to this movie in the similarity matrix. 
# Thus, we will get the similarity scores of all other movies from the current movie. 
# Then we will enumerate through all the similarity scores of that movie to make a tuple of movie index and similarity score. 
# This will convert a row of similarity scores like this- `[1 0.5 0.2 0.9]` to this- `[(0, 1) (1, 0.5) (2, 0.2) (3, 0.9)]` . 
# Here, each item is in this form- (movie index, similarity score).

In [11]:
movie_user_likes = "Pirates of the Caribbean: At World's End"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index])) 
#accessing the row corresponding to given movie to find all the similarity scores for that movie and then enumerating over it

In [12]:
# We will sort the list similar_movies according to similarity scores in descending order. 
# Since the most similar movie to a given movie will be itself, we will discard the first element after sorting the movies.
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]
sorted_similar_movies

[(199, 0.6207522318391884),
 (12, 0.6091095901015049),
 (5, 0.2556038601690775),
 (115, 0.2556038601690775),
 (30, 0.2506402059138015),
 (536, 0.23732222626728364),
 (1017, 0.23333333333333328),
 (129, 0.22841609628806428),
 (2029, 0.22841609628806428),
 (1652, 0.22360679774997896),
 (13, 0.21908902300206642),
 (271, 0.21908902300206642),
 (4211, 0.21081851067789195),
 (2474, 0.20942695414584772),
 (1708, 0.19674775073518586),
 (17, 0.19462473604038072),
 (178, 0.19462473604038072),
 (262, 0.19462473604038072),
 (894, 0.19034674690672024),
 (1095, 0.19034674690672024),
 (448, 0.1884336657203579),
 (329, 0.18633899812498247),
 (330, 0.18633899812498247),
 (840, 0.18633899812498247),
 (3374, 0.18633899812498247),
 (267, 0.18257418583505536),
 (785, 0.18257418583505536),
 (1031, 0.18257418583505536),
 (784, 0.17902871850985821),
 (1468, 0.17902871850985821),
 (2515, 0.17902871850985821),
 (1207, 0.1790287185098582),
 (1016, 0.17568209223157663),
 (3033, 0.17568209223157663),
 (3533, 0.175

In [13]:
i=0
print("Top 5 similar movies to \'"+movie_user_likes+"\' are:\n")
for element in sorted_similar_movies:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>5:
        break

Top 5 similar movies to 'Pirates of the Caribbean: At World's End' are:

Pirates of the Caribbean: The Curse of the Black Pearl
Pirates of the Caribbean: Dead Man's Chest
Spider-Man 3
Hancock
Spider-Man 2
Anna and the King
